# Get accelerator device

In [2]:
import torch

device = torch.accelerator.current_accelerator(
) if torch.accelerator.is_available() else 'cpu'
print(f'using device: {device}')

using device: mps


# PyTorch training loop

In [ ]:
import datetime

import torch
from torch import optim, nn
epoch_num = 1
train_loader = None  # torch dataset loader

model = None
learning_rate = 5e-6
optimizer = optim.AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss(reduction='none')

model.train()
for epoch in range(epoch_num):
    print(f'running epoch: {epoch}')
    for batch, (X, Y, loss_mask) in enumerate(train_loader):
        # compute prediction and loss
        X = X.to(device)
        Y = Y.to(device)
        loss_mask = loss_mask.to(device)
        pred = model(X)
        loss = loss_fn(pred.view(-1, pred.size(-1)), Y.view(-1)).view(Y.size())

        loss = (loss * loss_mask).sum() / loss_mask.sum()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        loss = loss.item()
        print(
            f"{datetime.datetime.now()}, epoch: {epoch}, step: {batch}, loss: {loss}"
        )